In [1]:
import json
import kickscore as ks
import numpy as np
import os
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split

In [2]:
_RANDOM_STATE = 5

# Data

In [3]:
comparisons = pickle.load(open('data/comparisons_berlin.p', 'rb'))
print(comparisons.shape)

(7281, 12)


In [4]:
images_list = pd.unique(comparisons[['image_l', 'image_r']].values.ravel('K'))

Split data into training and testing

In [5]:
X_train, X_test, = train_test_split(comparisons, test_size=0.15, random_state=_RANDOM_STATE, )# shuffle=False)
print('Train:', X_train.shape)
print('Test:', X_test.shape)

Train: (6188, 12)
Test: (1093, 12)


In [6]:
images = set()
observations = list()

for i, row in X_train.reset_index().iterrows():
    t = i
    images.add(row.image_l)
    images.add(row.image_r)
    
    if row.score == -1:
        observations.append({
                'winners': [row.image_l],
                'losers': [row.image_r],
                #'tie': False,
                't': t,
            })
    if row.score == 0:
        observations.append({
                'winners': [row.image_l],
                'losers': [row.image_r],
                'tie': True,
                't': t,
            })
    if row.score == 1:
        observations.append({
                'winners': [row.image_r],
                'losers': [row.image_l],
                #'tie': False,
                't': t,
            })

# Gaussian Process

## Model

In [7]:
model = ks.TernaryModel(margin=0.2)

## Kernel 

In [8]:
kernel = (ks.kernel.Constant(var=0.03))

## Set items

In [9]:
for image in images:
    model.add_item(image, kernel=kernel)

## Set comparisons

In [10]:
for obs in observations:
    model.observe(**obs)

## Train

In [11]:
%%time
converged = model.fit()
if converged:
    print("Model has converged.")

Model has converged.
CPU times: user 9.16 s, sys: 87.9 ms, total: 9.24 s
Wall time: 9.33 s


### Scores

In [12]:
ts = [comparisons.shape[0] ]  # Point in time at which you want to make the prediction.
res = dict()  # Contains predicted score.

scores = []
for name, item in model.item.items():
    means, var = item.predict(ts)
    scores += [[name, means[0], var[0]]]

scores = pd.DataFrame(scores, columns=['image', 'score', 'var'])

scores_df = scores.sort_values(by='image').reset_index(drop=True)
#for rank, (name, score) in enumerate(sorted(res.items(), key=lambda x: x[1], reverse=True), start=1):
#    print(f"rank {rank}: {name} (score: {score:.2f}")

In [13]:
scores_df = scores_df.set_index('image', drop=False)
scores_df.index.name = None

In [14]:
scores_df['image_path'] = scores_df['image']
scores_df['image_path'] = scores_df['image_path'].apply(lambda x: os.path.join('images','berlin', str(x) + '.jpg' ))

# Metrics

In [15]:
def compute_logloss(df):
    log_loss = []
    
    for i, row in df.iterrows():
        try:
            p_win, p_tie, p_los = model.probabilities([row.Winner], [row.Loser], t=comparisons.shape[0])

            if row.score == -1 or row.score == 1:
                log_loss.append(np.log(p_win))
            else:    
                log_loss.append(np.log(p_tie))  
        except KeyError:
            continue    
    return log_loss

In [16]:
def compute_accuracy(df):
    accuracy = []
    
    for i, row in df.iterrows():
        try:
            p_win, p_tie, p_los = model.probabilities([row.Winner], [row.Loser], t=comparisons.shape[0])

            if row.score == -1 or row.score == 1:
                if p_win > p_los and p_win > p_tie:
                    accuracy.append(1) 
                else:
                    accuracy.append(0) 
            else:  
                if p_tie > p_los and p_tie > p_win:
                    accuracy.append(1) 
                else:
                    accuracy.append(0) 
        except KeyError:
            continue
    return accuracy

### Compute metrics for training

In [17]:
log_loss_train = compute_logloss(X_train[X_train.score != 0])
accuracy_train = compute_accuracy(X_train[X_train.score != 0])

## Test

### Compute metrics for testing

In [18]:
log_loss_test = compute_logloss(X_test[X_test.score != 0])
accuracy_test = compute_accuracy(X_test[X_test.score != 0])

## Aggregate results

In [19]:
results = {
    'model': 'gaussianprocess',
    'train_logloss': -1 * np.mean(log_loss_train),
    'test_logloss': -1 * np.mean(log_loss_test),
    'train_accuracy': np.mean(accuracy_train),
    'test_accuracy': np.mean(accuracy_test),
    'seed': _RANDOM_STATE
}
pickle.dump(results,  open('output/{}_modelresults_SEED{}.p'.format(results['model'], _RANDOM_STATE), 'wb'))
print(json.dumps(results, indent=4))

{
    "model": "gaussianprocess",
    "train_logloss": 0.7966539920858491,
    "test_logloss": 0.8385269418911184,
    "train_accuracy": 0.9186555290373906,
    "test_accuracy": 0.6654411764705882,
    "seed": 5
}


In [20]:
pickle.dump(scores_df,  open('output/{}_scores_SEED{}.p'.format(results['model'], _RANDOM_STATE), 'wb'))